In [25]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.palettes import brewer
from bokeh.io import output_notebook

Load the GeoJSON file with LA Neighborhood Council Information:

In [26]:
nc_map = gpd.read_file('https://data.lacity.org/api/geospatial/fu65-dz2f?method=export&format=GeoJSON')

Get 311 data for 2019 and convert the 'CreatedDate' column into datetime:

In [27]:
df_311 = pd.read_csv('https://data.lacity.org/api/views/pvft-t768/rows.csv?accessType=DOWNLOAD')
df_311['CreatedDate1']=pd.to_datetime(df_311['CreatedDate']) 

Extract the needed info from the downloaded dataset and add in a 'Month' column

In [28]:
df_311 = df_311[['Latitude','Longitude','CreatedDate1','NCName']]
df_311['Month'] = df_311['CreatedDate1'].dt.month
df_311['NCName'] = df_311['NCName'].astype(str)

Get the Call Volume for May 2019

In [29]:
may311_df = df_311.loc[(df_311['Month'] == 5)].groupby('NCName').agg('count')
may311_df = may311_df.reset_index()
may311_df.rename({'NCName':'name','CreatedDate1':'Calls'},axis=1,inplace=True)
may311_df.drop(['Latitude','Longitude','Month'],axis=1,inplace=True)

Merge the geopanda file with the May 2019 call data

In [30]:
nc_calls = nc_map.merge(may311_df,on='name')

Define a function, getPolyCoords, to be used to extract x and y coordinate data from nc_calls

In [31]:
def getPolyCoords(row, geom, coord_type):
    exterior = row[geom][0].exterior

    if coord_type == 'x':
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        return list( exterior.coords.xy[1] )

In [32]:
nc_calls['x'] = nc_calls.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
nc_calls['y'] = nc_calls.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

Convert nc_calls into a ColumnDataSource object

In [33]:
nc_calls.drop('geometry',axis=1,inplace=True)

In [34]:
nsource = ColumnDataSource(nc_calls)

Intialize figure and the tools it will use

In [35]:
TOOLTIPS = [('Neighborhood Council Name','@name'),('Call Volume','@Calls')]

In [36]:
p1 = figure(title="May 2019 311 Call Volume by Neighborhood Council",tools='hover, pan, wheel_zoom',tooltips=TOOLTIPS)

Set the color scheme and the renderer

In [37]:
palette = brewer['YlGnBu'][8]
palette = palette[::-1]

In [38]:
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 3000)

In [39]:
p1.patches('x', 'y', source=nsource,fill_color = {'field' :'Calls', 'transform' : color_mapper},
        line_color="black", line_width=0.05)

GlyphRenderer(id='1490', ...)

In [40]:
output_notebook()

Loading BokehJS ...

In [42]:
show(p1)